In [1]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
import numpy as np
import scipy.sparse as scps
import scipy.sparse.linalg as ssl
import math
from mpl_toolkits.mplot3d import Axes3D

ArgumentError: ArgumentError: Package matplotlib not found in current path.
- Run `import Pkg; Pkg.add("matplotlib")` to install the matplotlib package.

In [2]:
def maillage(n):
#
# Une discretisation possible d'une EDP elliptique sur le domaine ]0,1[ x ]0,1[
# Le carre [0,1]x[0,1] est maille uniquement avec des triangles; 
# Les conditions limites sont de type Dirichlet uniquement   => neumann  =[];
#
# Entrees :
# n : nombre de points par cote du care => Npts points de discretisation au
# total
#
# Sorties :
# coordinates : matrice a deux colonnes. Chaque ligne contient les 
# coordonnes 2D d'un des points de la discretisation. Ces sommets seront 
# identifies a l'indice de la ligne correspondante dans la matrice
# coordinates.
# elements3 : matrice a trois colonnes. Chaque ligne contient les indices 
# des sommets d'un element triangle, dans le sens antihoraire. 
# dirichlet : vecteur colonne des indices des sommets de la frontiere de
# Dirichlet.
# neumann : matrice a deux colonnes. Chaque ligne contient les indices 
# des deux sommets d'une arete de la frontiere de Neumann.
# (neumann est vide sur cet exemple)
#
##################################################################################
    h=1/(n-1)
    npoin       = n*n ; 
    nelem       = 2*(n-1)*(n-1) ;
    coordinates = np.zeros((npoin,2)); 
    elements3   = (np.zeros((nelem,3))).astype(int) ;
    neumann     = [];
    dirichlet=(np.zeros((4*n-4,1))).astype(int)
    # Coordonnees et connectivites :
    e = -1 ; 
    p = -1 ;
    x=np.zeros((n+1,1))
    x[n,0]=1.
    for l in range (n+1):
        x[l,0]=l*h
    for j in range (n):
            for i in range(n):
                p = p + 1  
                coordinates[p,0] = x[i,0]  
                coordinates[p,1] = x[j,0] 
                if ((i != n-1) & (j != n-1)):
                    p1 = p
                    p2 = p1 + 1 
                    p3 = p1 + n 
                    p4 = p2 + n 
                    e = e + 1 
                    elements3[e,0] = p1 
                    elements3[e,1] = p2 
                    elements3[e,2] = p3 
                    e = e + 1
                    elements3[e,0] = p4 
                    elements3[e,1] = p3 
                    elements3[e,2] = p2 
    #Liste des sommets de la frontiere de Dirichlet:
    p=-1
    for j in range(n):
        p=p+1
        dirichlet[p,0] = j  
    for j in range(n*2-1,n*(n-1),n):
        p=p+1
        dirichlet[p,0] = j 
    for j in range(n*n-1,n*n-n-1,-1):
        p=p+1
        dirichlet[p,0] = j 
    for j in range(n*n-2*n,n-1,-n):
        p=p+1
        dirichlet[p,0] = j 

    return coordinates, elements3,dirichlet, neumann

ErrorException: syntax: extra token "maillage" after end of expression

In [3]:
def show(coordinates,u):
#
# Fonction d'affichage de la solution u sur le maillage defini par
# elements3, coordinates.
#
# Entrees:
# elements3 : matrice a trois colonnes contenant les elements triangles
# de la discretisation, identifies par les indices de leurs trois
# sommets.
# coordinates : matrice a deux colonnes contenant les coordonnes 2D des
# points de la discretisation.
# u : vecteur colonne de longueur egale au nombre de lignes de
# coordinates contenant les valeurs de la solution a afficher aux
# points de la discretisation.
#
# Sorties : Aucune, mais la fonction doit s'afficher dans une figure.
##########################################################################
    ax= plt.figure().add_subplot(projection='3d')
    ax.plot_trisurf(coordinates[:,0],coordinates[:,1],u,linewidth=0.2,antialiased=True)
    plt.show()

ErrorException: syntax: extra token "show" after end of expression

**Partie I : maillage triangulaire et conditions de Dirichlet**

In [4]:

'''
def compute_MTA():
    # Construction de la matrice de raideur élémentaire MTA relative à un élément triangle
    
    # Coordonnées des sommets de l'élément triangle dans le plan (x,y)
    x = np.array([0, 1, 0])
    y = np.array([0, 0, 1])
    
    # Calcul des gradients des fonctions de base de l'élément triangle
    J = np.array([[1, 1, 1], [x[0], x[1], x[2]], [y[0], y[1], y[2]]])
    detJ = np.linalg.det(J)
    invJ = np.linalg.inv(J)
    G = np.matmul(invJ, np.array([[0, 1, 0], [0, 0, 1], [1, 1, 1]]))
    
    # Calcul de la matrice de raideur élémentaire MTA
    MTA = np.matmul(np.transpose(G), np.matmul(np.array([[1, -1, 0], [-1, 1, 0], [0, 0, 0]]), G)) * detJ / 2
    
    return MTA '''

def assemble_A(coordinates, elements3):
    # Assemblage de la matrice A dans le cas d'un maillage constitué uniquement d'éléments triangles
    
    npoin = len(coordinates)
    nelem = len(elements3)
    A = np.zeros((npoin, npoin))
    
    for t in elements3:
        grads = np.empty((3,), dtype=object)
        p1,p2,p3 = coordinates[t[0]],coordinates[t[1]],coordinates[t[2]]
        alpha = (p2[0]-p1[0])*(p3[1]-p1[1]) - (p3[0] - p1[0])*(p2[1]- p1[1])
        grad[0] = np.array([(p2[1]-p3[1]),(p3[1]-p1[1]),(p1[1]-p2[1])])/(2*alpha)
        grad[1] = np.array([(p3[0]-p2[0]),(p1[0]-p3[0]),(p2[0]-p1[0])])/(2*alpha)
        for i in range(3):
            for j in range(3):
                A[t[i],t[j]] += np.dot(grad[i],grad[j])
    return A

def assemblage_seconde_membre(coordinates, elements3, dirichlet,ud,f):
    npoin = np.size(coordinates[:,0])
    ntriang = np.size(elements3[:,0])
    A = assemble_A(coordinates,elements3)
    b = np.zeros(npoin)
    for t in elements3:
        #trouver le barycentre
        p1,p2,p3 = coordinates[t[0]],coordinates[t[1]],coordinates[t[2]]
        pg = (1/3)*(p1+p2+p3)
        alpha = (p2[0]-p1[0])*(p3[1]-p1[1]) - (p3[0] - p1[0])*(p2[1]- p1[1])
        #calcule de alpha/6 * f(xg,yg)
        abc = np.linalg.solve(np.array([[p1[0],p1[1],1.],[p2[0],p2[1],1.],[p3[0],p3[1],1.]]),np.array([f[t[0]],f[t[1]],f[t[2]]]))
        fxg = abc[0]*pg[0] + abc[1]*pg[1] + abc[2]
        b[t[0]] += (alpha/6)*fxg
        b[t[1]] += (alpha/6)*fxg
        b[t[2]] += (alpha/6)*fxg
        #####################################################
        for p in dirichlet:
            xyp = coordinates[p][0]
            for i in range(3):     
                if (t[i] in dirichlet) : 
                    if ( (t[i] == 0 ) or (t[i] == npoin - 1) ):
                        b[t[i]] -= ud[p[0]]*A[t[i],p]
                    elif ( (t[i] == int(np.sqrt(npoin)) -1 ) or (t[i] == npoin*npoin - npoin ) ):
                        b[t[i]] -= (1/2)*ud[p[0]]*A[t[i],p]
                    else :
                        b[t[i]] -= (1/3)*ud[p[0]]*A[t[i],p]
                else :
                        b[t[i]] -= (1/6)*ud[p[0]]*A[t[i],p]
            
    return b
        #############################################################""

            
            

    

ErrorException: syntax: extra token "compute_MTA" after end of expression

In [5]:
coordinates, elements3,dirichlet, neumann = maillage(10)   
f = np.cos(np.linspace(0.,np.pi,100))
ud = np.ones((1,100))
for t in elements3:
        #trouver le barycentre
        p1,p2,p3 = coordinates[t[0]],coordinates[t[1]],coordinates[t[2]]
        pg = (1/3)*(p1+p2+p3)
        alpha = (p2[0]-p1[0])*(p3[1]-p1[1]) - (p3[0] - p1[0])*(p2[1]- p1[1])
        #calcule de alpha/6 * f(xg,yg)
        abc = np.linalg.solve(np.array([[p1[0],p1[1],1.],[p2[0],p2[1],1.],[p3[0],p3[1],1.]]),np.array([f[t[0]],f[t[1]],f[t[2]]]))
        print(abc)

UndefVarError: UndefVarError: maillage not defined

In [6]:


# Définition des paramètres du maillage
n = 10
'''
# Génération du maillage
coordinates, elements3, dirichlet, neumann = maillage(n)

# Assemblage du second membre
f = assemblage_seconde_membre(coordinates, elements3, dirichlet)

# Affichage du second membre
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(coordinates[:, 0], coordinates[:, 1], f[:, 0])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f')
plt.show()
'''


UndefVarError: UndefVarError: maillage not defined

**Partie II : maillage mixte et ajoût des conditions de Neumann**

In [8]:
############################# Maillage mixte ################
e3=np.array([[1,2,12],[2,3,12],[3,4,14],[4,5,14],[2,15,3],[3,15,4]]).astype(int)
e4=np.array([[0,1,12,11],[11,12,13,10],[12,3,14,13],[10,13,8,9],[13,14,7,8],[14,5,6,7]]).astype(int)
dds=np.array([2,15,4,6,7,8,9,10,11,0]).astype(int)
nns=np.array([[4,5],[5,6],[0,1],[1,2]]).astype(int)
ccs=np.array([[0.,0.],[0.33333333333333,0],[0.53333333333333,0.],
                      [0.66666666666667,0.33333333333333],[1.,0.47],[1,0.66666666666667],
                     [1.,1.],[0.66666666666667,1.],[0.33333333333333,1.], [0.,1.],
                     [0.,0.66666666666667],[0.,0.33333333333333],[0.33333333333333,0.33333333333333],
                     [0.33333333333333,0.66666666666667],[0.66666666666667,0.66666666666667],[1.,0.]])

UndefVarError: UndefVarError: np not defined

In [9]:
coordinates, elements3,dirichlet, neumann = maillage(5)
n = 25
A = assemble_A(coordinates,elements3)
b = assemblage_seconde_membre(coordinates,elements3,dirichlet,np.ones((n)),np.cos(np.linspace(0.,np.pi,n)))
print(b)


UndefVarError: UndefVarError: maillage not defined

**Compléments  :  un nouveau terme dans l'EDP**